In [1]:
import findspark
findspark.init('C:\Spark\spark-3.0.1-bin-hadoop2.7')
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [3]:
from pyspark.ml.classification import LogisticRegression

In [4]:
data = spark.read.csv('titanic.csv',
                      inferSchema=True,
                      header=True)

In [5]:
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [6]:
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [7]:
my_cols = data.select(['Survived',
 'Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Fare',
 'Embarked'])

In [8]:
from pyspark.sql.functions import isnan, isnull, when, count, col

my_cols.select([count(when(isnan(c)| isnull(c), c)).alias(c) for c in my_cols.columns]).show()

+--------+------+---+---+-----+-----+----+--------+
|Survived|Pclass|Sex|Age|SibSp|Parch|Fare|Embarked|
+--------+------+---+---+-----+-----+----+--------+
|       0|     0|  0|177|    0|    0|   0|       2|
+--------+------+---+---+-----+-----+----+--------+



In [9]:
data = my_cols.na.drop()

In [10]:
data.printSchema()

root
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Embarked: string (nullable = true)



In [11]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [12]:
sex_indexer = StringIndexer(inputCol='Sex', outputCol='SexIndex')
sex_encoder = OneHotEncoder(inputCol='SexIndex', outputCol='SexVec')
embarked_indexer = StringIndexer(inputCol='Embarked', outputCol='EmbarkIndex')
embarked_encoder = OneHotEncoder(inputCol='EmbarkIndex', outputCol='EmbarkVec')

In [13]:
assembler = VectorAssembler(
    inputCols=['Pclass',
               'SexVec',
               'Age',
               'SibSp',
               'Parch',
               'Fare',
               'EmbarkVec'],
    outputCol='features'
)

In [14]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [15]:
log_reg_titanic = LogisticRegression(
    featuresCol='features',
    labelCol='Survived'
)

In [16]:
pipeline = Pipeline(stages=[sex_indexer,embarked_indexer,
                            sex_encoder,embarked_encoder,
                            assembler,log_reg_titanic])

In [17]:
train, test = data.randomSplit([0.7,.3])

In [18]:
model = pipeline.fit(train)

In [19]:
results = model.transform(test)

In [20]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [21]:
evaluator = BinaryClassificationEvaluator(
    rawPredictionCol='prediction',
    labelCol='Survived'
)

In [22]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [23]:
evaluator.evaluate(results)

0.7845585794888815